In [1]:
import json
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

# Function to calculate IoU score
def calculate_iou(string1, string2):
    # Tokenize both strings using spaCy
    tokens1 = set(token.text for token in nlp(string1))
    tokens2 = set(token.text for token in nlp(string2))

    # Calculate intersection and union of tokens
    intersection = len(tokens1 & tokens2)
    union = len(tokens1 | tokens2)

    # Calculate IoU score
    iou_score = intersection / union

    return iou_score

# Example strings
string1 = "This is an example sentence."
string2 = "This is another example."

# Calculate IoU score
iou_score = calculate_iou(string1, string2)

print(f"IoU Score: {iou_score:.2f}")

IoU Score: 0.57


In [276]:
with open('/content/GPT35_test_qa_parsed.json', 'r') as f:
        gen_data = json.load(f)

In [251]:
with open('/content/test2.json', 'r') as f:
        gold_data = json.load(f)

In [259]:
num_match=0
num_gold=0
context_quest = []
score_dict={}
for key,value in gen_data.items():
  for k,v in gold_data.items():
    if(key==k):
      for event in value["events"]:
        for key_eve, val_eve in event.items():
          for item in v["events"]:
            if (key_eve==item["event_id"]):
              qa_pairs_gold = item["manual"]
              score_dict[item["event_id"]]=[]
              num_gold+=len(qa_pairs_gold)
              for qa_pair_gen in val_eve:
                iou_sc = []
                temp_dict={}
                for qa_pair in qa_pairs_gold:
                  val = calculate_iou(qa_pair_gen["question"],qa_pair["question"])
                  iou_sc.append(val)
                # print(iou_sc)
                if(iou_sc!=[]):
                    # print(qa_pair_gen["question"])
                    # print(qa_pair["question"])

                    max_iou_idx = iou_sc.index(max(iou_sc))
                    temp_dict[(qa_pair_gen["question"],qa_pairs_gold[max_iou_idx]["question"])]=max(iou_sc)
                    score_dict[item["event_id"]].append(temp_dict)

for key,value in score_dict.items():
  for item in value:
    for k,v in item.items():
      if v>0:
        num_match+=1



In [260]:
print(num_match)

97


In [254]:
print(num_gold)

113


In [201]:
print(score_dict)

{'scenario_en_kairos_68-E1': [{("What event triggered the speaker's apology?", 'What religion is the bomber?'): 0.2727272727272727}, {('Who expressed guilt for the bombing?', 'Who bombed?'): 0.25}, {('Did the speaker act alone in the bombing?', 'What religion is the bomber?'): 0.16666666666666666}, {('Who did the speaker apologize to for the bombing?', 'Who bombed?'): 0.2}, {('What kind of damage did the speaker admit to causing?', 'What religion is the bomber?'): 0.21428571428571427}], 'wiki_drone_strikes_0_news_1-E1': [], 'wiki_drone_strikes_0_news_1-E2': [], 'wiki_drone_strikes_0_news_1-E3': [], 'wiki_drone_strikes_0_news_1-E4': [], 'wiki_drone_strikes_0_news_1-E5': [{('Who was killed?', 'How many people did the us estimate was killed'): 0.18181818181818182}, {('How many civilians were killed in Pakistan, Somalia, and Yemen?', 'How many people did the us estimate was killed'): 0.16666666666666666}, {('How many civilians were killed in Afghanistan in 2016?', 'How many people did the 

In [80]:
print(f"GPT QAQA:{num_match}")

GPT QAQA:455


In [83]:
print(f"Rule_Gold QG:{num_match}")

Rule_Gold QG:218


In [88]:
print(f"Rule_Pred QG:{num_match}")

Rule_Pred QG:185


In [77]:
print(f"GPT AQAQ:{num_match}")

GPT AQAQ:482


In [184]:
print(len(context_quest)) #These many pairs were considered

198


In [329]:
#SENTENCE WISE CONTEXT
with open('/content/GPT4_QA.json', 'r') as f:
        gen_data = json.load(f)
with open('/content/test2.json', 'r') as f:
        gold_data = json.load(f)

In [337]:
num_match=0
num_ques=0
num_gold=0
context_quest = []
score_dict={}
for key,value in gen_data.items():
  for k,v in gold_data.items():
    if(key==k):
      doc=v["text"]
      for event in value["events"]:
        for key_eve, val_eve in event.items():
          for item in v["events"]:
            if (key_eve==item["event_id"]):
              qa_pairs_gold = item["manual"]
              score_dict[item["event_id"]]=[]
              trigger = item["trigger"]
              sentences = doc.split('. ')
              context = None
              for i, sentence in enumerate(sentences):
                if trigger in sentence:
                  context = sentence
                  break
              for qa_pair_gen in val_eve:
                iou_sc = []
                temp_dict={}
                # gold_qa=0
                try:
                  if(qa_pair_gen["answer"] in context):
                    for qa_pair in qa_pairs_gold:
                      if(qa_pair["answer"] in context):
                        num_gold+=1
                        val = calculate_iou(qa_pair_gen["question"],qa_pair["question"])
                        iou_sc.append(val)
                    # print(iou_sc)
                    if(iou_sc!=[]):
                      # num_gold+=len(iou_sc)
                      max_iou_idx = iou_sc.index(max(iou_sc))
                      temp_dict[(qa_pair_gen["question"],qa_pairs_gold[max_iou_idx]["question"])]=max(iou_sc)
                      score_dict[item["event_id"]].append(temp_dict)
                except Exception as e:
                  continue
for key,value in score_dict.items():
  for item in value:
    for k,v in item.items():
      if v>0:
        num_match+=1

In [338]:
print(num_match)
# print(num_gold)

117


In [332]:
print(score_dict)

{'scenario_en_kairos_68-E1': [], 'wiki_drone_strikes_0_news_1-E1': [], 'wiki_drone_strikes_0_news_1-E2': [], 'wiki_drone_strikes_0_news_1-E3': [], 'wiki_drone_strikes_0_news_1-E4': [], 'wiki_drone_strikes_0_news_1-E5': [], 'wiki_drone_strikes_0_news_1-E6': [], 'wiki_drone_strikes_0_news_1-E7': [], 'wiki_drone_strikes_0_news_1-E8': [], 'wiki_drone_strikes_0_news_1-E9': [], 'wiki_drone_strikes_0_news_1-E10': [], 'wiki_drone_strikes_0_news_1-E11': [], 'wiki_drone_strikes_0_news_1-E12': [{('What is happening to Afghanistan?', 'Which country came under US bombardment'): 0.25}, {('How many weapons were dropped in Afghanistan in 2016?', 'Which country came under US bombardment'): 0.08333333333333333}], 'wiki_drone_strikes_0_news_1-E13': [], 'wiki_drone_strikes_0_news_1-E14': [], 'wiki_drone_strikes_0_news_1-E15': [], 'wiki_drone_strikes_0_news_1-E16': [], 'wiki_drone_strikes_0_news_1-E17': [], 'wiki_drone_strikes_0_news_1-E18': [], 'wiki_ied_bombings_10-E1': [], 'wiki_ied_bombings_10-E2': [],

In [282]:
#NUM Q context based
num_q=0
for k,v in gold_data.items():
  doc=v["text"]
  for item in v["events"]:
    qa_pairs_gold = item["manual"]
    trigger = item["trigger"]
    sentences = doc.split('. ')
    context = None
    for i, sentence in enumerate(sentences):
      if trigger in sentence:
        context = sentence
      break
    try:
      for qa_pair in qa_pairs_gold:
        if(qa_pair["answer"] in context):
            num_q+=1
    except Exception as e:
      continue

In [283]:
print(num_q)

39



# **ANSWER_EVALUATION**

In [368]:
def clean_mention(text):
    """
    Clean up a mention by removing ‘a’, ‘an’, ‘the’ prefixes.
    """
    prefixes = ['the', 'The', 'an', 'An', 'a ', 'A ']
    for prefix in prefixes:
        if text.startswith(prefix):
            return text[len(prefix):]
    return text

In [379]:
import sys
import json
import re
import string
from collections import Counter


In [380]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [410]:
def f1_score(prediction, ground_truth):
    normalized_prediction = normalize_answer(prediction)
    normalized_ground_truth = normalize_answer(ground_truth)

    ZERO_METRIC = (0, 0, 0)

    # if normalized_prediction != normalized_ground_truth:
    #     print("Return from HERE!!!!")
    #     return ZERO_METRIC

    prediction_tokens = normalized_prediction.split()
    ground_truth_tokens = normalized_ground_truth.split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        # print("Return from HERE!!!!!!!!!!!!!!!!")
        return ZERO_METRIC
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

In [411]:
def update_answer(metrics, prediction, gold):
    f1, prec, recall = f1_score(prediction, gold)
    metrics['f1'] += f1
    metrics['prec'] += prec
    metrics['recall'] += recall

In [417]:
metrics = {'f1': 0, 'prec': 0, 'recall': 0}


In [418]:
import json
import spacy
i=0  #to average out the result
nlp = spacy.load("en_core_web_sm")
with open('/content/ans_id_test.json', 'r') as f:
        data = json.load(f)
for key,value in data.items():
  for event in value["events"]:
    qa_triplets=event["manual"]
    for qa_triplet in qa_triplets:
      pred_ans = qa_triplet["answer"]
      gold_ans = qa_triplet["gold_ans"]
      i+=1
      update_answer(metrics,pred_ans,gold_ans)
print(i)
print(f"F1: {metrics['f1']/i}\n")
print(f"Precision: {metrics['prec']/i}\n")
print(f"Recall: {metrics['recall']/i}\n")

132
F1: 0.49587242872330917

Precision: 0.47113887313795805

Recall: 0.6271329365079364



In [386]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")
with open('/content/ans_id_test_nodemo.json', 'r') as f:
        data = json.load(f)
num_match=0
num_ques=0
for key,value in data.items():
  for event in value["events"]:
    qa_triplets=event["manual"]
    num_ques+=len(qa_triplets)
    for qa_triplet in qa_triplets:
      pred_ans = qa_triplet["answer"]
      gold_ans = qa_triplet["gold_ans"]
      #cleanup
      pred_ans = clean_mention(pred_ans)
      gold_ans = clean_mention(gold_ans)
      tokens1 = set(token.text for token in nlp(pred_ans))
      tokens2 = set(token.text for token in nlp(gold_ans))
      match_found = False
      for token1 in tokens1:
        for token2 in tokens2:
            if token1 == token2:
                match_found = True
                break
        if match_found:
          break
      if match_found:
        num_match += 1

print(f"OVERLAP:{num_match}")
print(num_ques)

OVERLAP:108
132


In [374]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")
with open('/content/ans_id_test_nodemo.json', 'r') as f:
        data = json.load(f)
num_match=0
num_ques=0
match_found=False
for key,value in data.items():
  for event in value["events"]:
    qa_triplets=event["manual"]
    num_ques+=len(qa_triplets)
    for qa_triplet in qa_triplets:
      pred_ans = qa_triplet["answer"]
      gold_ans = qa_triplet["gold_ans"]
      #cleanup
      pred_ans = clean_mention(pred_ans)
      gold_ans = clean_mention(gold_ans)
      tokens1=nlp(pred_ans)
      tokens2=nlp(gold_ans)
      for np1,np2 in zip(tokens1.noun_chunks,tokens2.noun_chunks):
        if(str(np1.root)==str(np2.root)):
          num_match+=1
print(f"NOUN HEAD:{num_match}")
print(num_ques)

NOUN HEAD:99
132


In [375]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")
with open('/content/ans_id_test_nodemo.json', 'r') as f:
        data = json.load(f)
num_match=0
num_ques=0
match_found=False
for key,value in data.items():
  for event in value["events"]:
    qa_triplets=event["manual"]
    num_ques+=len(qa_triplets)
    for qa_triplet in qa_triplets:
      pred_ans = qa_triplet["answer"]
      gold_ans = qa_triplet["gold_ans"]
      #cleanup
      pred_ans = clean_mention(pred_ans)
      gold_ans = clean_mention(gold_ans)
      doc1 = nlp(pred_ans)
      doc2 = nlp(gold_ans)
      normalized_tokens1 = [token.text.lower() for token in doc1 if not token.is_punct and not token.is_space]
      normalized_tokens2 = [token.text.lower() for token in doc2 if not token.is_punct and not token.is_space]
      if (set(normalized_tokens1) == set(normalized_tokens2)):
        num_match+=1
print(f"EM:{num_match}")
print(num_ques)

EM:32
132


In [376]:
import json
import spacy

nlp = spacy.load("en_core_web_sm")
with open('/content/ans_id_test_nodemo.json', 'r') as f:
        data = json.load(f)
num_match=0
num_ques=0
match_found=False
for key,value in data.items():
  for event in value["events"]:
    qa_triplets=event["manual"]
    num_ques+=len(qa_triplets)
    for qa_triplet in qa_triplets:
      pred_ans = qa_triplet["answer"]
      gold_ans = qa_triplet["gold_ans"]
      #cleanup
      pred_ans = clean_mention(pred_ans)
      gold_ans = clean_mention(gold_ans)
      if(pred_ans==gold_ans):
        num_match+=1
print(f"EM_String:{num_match}")
print(num_ques)

EM_String:20
132
